In [1]:
from transformers import AutoModel, AutoTokenizer
from gliner import GLiNER
import torch

/Users/artemon/.local/share/virtualenvs/models-research-YeHw3bbB/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = GLiNER.from_pretrained('numind/NuNerZero')

/Users/artemon/.local/share/virtualenvs/models-research-YeHw3bbB/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/Users/artemon/.local/share/virtualenvs/models-research-YeHw3bbB/lib/python3.11/site-packages/gliner/model.py:420: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for mo

In [3]:
model.save_pretrained('./dump/dummy_model')

In [19]:
text = 'Hellow fedorov artem, how are you?'
labels = ['name', 'are']
tokens = model.token_splitter(text)
[*tokens]

[('Hellow', 0, 6),
 ('fedorov', 7, 14),
 ('artem', 15, 20),
 (',', 20, 21),
 ('how', 22, 25),
 ('are', 26, 29),
 ('you', 30, 33),
 ('?', 33, 34)]

In [20]:
model.predict_entities(text, labels)

[{'start': 7,
  'end': 14,
  'text': 'fedorov',
  'label': 'name',
  'score': 0.9380099773406982},
 {'start': 15,
  'end': 20,
  'text': 'artem',
  'label': 'name',
  'score': 0.9013848900794983}]

In [21]:
def merge_entities(entities):
    if not entities:
        return []
    merged = []
    current = entities[0]
    for next_entity in entities[1:]:
        if next_entity['label'] == current['label'] and (next_entity['start'] == current['end'] + 1 or next_entity['start'] == current['end']):
            current['text'] = text[current['start']: next_entity['end']].strip()
            current['end'] = next_entity['end']
        else:
            merged.append(current)
            current = next_entity
    # Append the last entity
    merged.append(current)
    return merged


# NuZero requires labels to be lower-cased!
# labels = ["organization", "initiative", "project"]
# labels = [l.lower() for l in labels]

# text = "At the annual technology summit, the keynote address was delivered by a senior member of the Association for Computing Machinery Special Interest Group on Algorithms and Computation Theory, which recently launched an expansive initiative titled 'Quantum Computing and Algorithmic Innovations: Shaping the Future of Technology'. This initiative explores the implications of quantum mechanics on next-generation computing and algorithm design and is part of a broader effort that includes the 'Global Computational Science Advancement Project'. The latter focuses on enhancing computational methodologies across scientific disciplines, aiming to set new benchmarks in computational efficiency and accuracy."

entities = model.predict_entities(text, labels)

entities = merge_entities(entities)

for entity in entities:
    print(entity["text"], "=>", entity["label"])


fedorov artem => name


In [22]:
import inspect

In [25]:
print(inspect.getsource(model.batch_predict_entities))

    def batch_predict_entities(self, texts, labels, flat_ner=True, threshold=0.5, multi_label=False):
        """
        Predict entities for a batch of texts.
        texts:  List of texts | List[str]
        labels: List of labels | List[str]
        ...
        """

        all_tokens = []
        all_start_token_idx_to_text_idx = []
        all_end_token_idx_to_text_idx = []

        for text in texts:
            tokens = []
            start_token_idx_to_text_idx = []
            end_token_idx_to_text_idx = []
            for token, start, end in self.token_splitter(text):
                tokens.append(token)
                start_token_idx_to_text_idx.append(start)
                end_token_idx_to_text_idx.append(end)
            all_tokens.append(tokens)
            all_start_token_idx_to_text_idx.append(start_token_idx_to_text_idx)
            all_end_token_idx_to_text_idx.append(end_token_idx_to_text_idx)

        input_x = [{"tokenized_text": tk, "ner": None} for tk in all_

In [29]:
[*model.token_splitter(text)]

[('Hellow', 0, 6),
 ('fedorov', 7, 14),
 ('artem', 15, 20),
 (',', 20, 21),
 ('how', 22, 25),
 ('are', 26, 29),
 ('you', 30, 33),
 ('?', 33, 34)]

In [30]:
texts = [text]

In [31]:
all_tokens = []
all_start_token_idx_to_text_idx = []
all_end_token_idx_to_text_idx = []

for text in texts:
    tokens = []
    start_token_idx_to_text_idx = []
    end_token_idx_to_text_idx = []
    for token, start, end in model.token_splitter(text):
        tokens.append(token)
        start_token_idx_to_text_idx.append(start)
        end_token_idx_to_text_idx.append(end)
    all_tokens.append(tokens)
    all_start_token_idx_to_text_idx.append(start_token_idx_to_text_idx)
    all_end_token_idx_to_text_idx.append(end_token_idx_to_text_idx)


In [34]:
input_x = [{"tokenized_text": tk, "ner": None} for tk in all_tokens]

In [35]:
all_tokens, all_start_token_idx_to_text_idx, all_end_token_idx_to_text_idx, input_x

([['Hellow', 'fedorov', 'artem', ',', 'how', 'are', 'you', '?']],
 [[0, 7, 15, 20, 22, 26, 30, 33]],
 [[6, 14, 20, 21, 25, 29, 33, 34]],
 [{'tokenized_text': ['Hellow',
    'fedorov',
    'artem',
    ',',
    'how',
    'are',
    'you',
    '?'],
   'ner': None}])

In [111]:
labels = ['name', 'city location far away from home']

In [112]:
x = model.collate_fn(input_x, labels)
x

{'seq_length': tensor([8]),
 'span_idx': tensor([[[0, 0],
          [1, 1],
          [2, 2],
          [3, 3],
          [4, 4],
          [5, 5],
          [6, 6],
          [7, 7]]]),
 'tokens': [['Hellow', 'fedorov', 'artem', ',', 'how', 'are', 'you', '?']],
 'span_mask': tensor([[True, True, True, True, True, True, True, True]]),
 'span_label': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]),
 'entities': [None],
 'classes_to_id': {'name': 1, 'city location far away from home': 2},
 'id_to_classes': {1: 'name', 2: 'city location far away from home'}}

In [106]:
outputs = model.predict(x, flat_ner=True, threshold=0.5, multi_label=True)

In [107]:
outputs

[[(1, 1, 'name', 0.8806719183921814), (2, 2, 'name', 0.8604023456573486)]]

In [39]:
print(inspect.getsource(model.predict))

    @torch.no_grad()
    def predict(self, x, flat_ner=False, threshold=0.5, multi_label=False):
        self.eval()
        local_scores = self.compute_score_eval(x, device=next(self.parameters()).device)
        probs = torch.sigmoid(local_scores)

        spans = []
        for i, _ in enumerate(x["tokens"]):
            probs_i = probs[i]
            
            wh_i = [i.tolist() for i in torch.where(probs_i > threshold)]
            span_i = []
            
            for s, k, c in zip(*wh_i):
                if s + k < len(x["tokens"][i]):
                    span_i.append((s, s + k, x["id_to_classes"][c + 1], probs_i[s, k, c].item()))
            
            span_i = greedy_search(span_i, flat_ner, multi_label=multi_label)
            spans.append(span_i)
        return spans



In [113]:
x

{'seq_length': tensor([8]),
 'span_idx': tensor([[[0, 0],
          [1, 1],
          [2, 2],
          [3, 3],
          [4, 4],
          [5, 5],
          [6, 6],
          [7, 7]]]),
 'tokens': [['Hellow', 'fedorov', 'artem', ',', 'how', 'are', 'you', '?']],
 'span_mask': tensor([[True, True, True, True, True, True, True, True]]),
 'span_label': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]),
 'entities': [None],
 'classes_to_id': {'name': 1, 'city location far away from home': 2},
 'id_to_classes': {1: 'name', 2: 'city location far away from home'}}

In [41]:
print(inspect.getsource(model.compute_score_eval))

    def compute_score_eval(self, x, device):
        # check if classes_to_id is dict
        assert isinstance(x["classes_to_id"], dict), "classes_to_id must be a dict"

        span_idx = (x["span_idx"] * x["span_mask"].unsqueeze(-1)).to(device)

        all_types = list(x["classes_to_id"].keys())
        # multiple entity types in all_types. Prompt is appended at the start of tokens
        entity_prompt = []

        # add enity types to prompt
        for entity_type in all_types:
            entity_prompt.append(self.entity_token)
            entity_prompt.append(entity_type)

        entity_prompt.append(self.sep_token)

        prompt_entity_length = len(entity_prompt)

        # add prompt
        tokens_p = [entity_prompt + tokens for tokens in x["tokens"]]
        seq_length_p = x["seq_length"] + prompt_entity_length

        out = self.token_rep_layer(tokens_p, seq_length_p)

        word_rep_w_prompt = out["embeddings"]
        mask_w_prompt = out["mask"]

        # remove

In [45]:
model.entity_token, model.sep_token

('<<ENT>>', '<<SEP>>')

In [108]:
out = model.token_rep_layer([['<<ENT>>', 'name', 'hellow', '<<ENT>>', 'location', '<<SEP>>', 'Hellow', 'fedorov', 'artem']], torch.tensor([21]))

out['embeddings'].shape, out['mask'].shape

(torch.Size([1, 9, 768]), torch.Size([1, 21]))

In [100]:
inspect.findsource(model.prompt_rep_layer)

TypeError: module, class, method, function, traceback, frame, or code object was expected, got Sequential

In [114]:
# check if classes_to_id is dict
assert isinstance(x["classes_to_id"], dict), "classes_to_id must be a dict"

span_idx = (x["span_idx"] * x["span_mask"].unsqueeze(-1)).to('cpu')

all_types = list(x["classes_to_id"].keys())
# multiple entity types in all_types. Prompt is appended at the start of tokens
entity_prompt = []

# add enity types to prompt
for entity_type in all_types:
    entity_prompt.append(model.entity_token)
    entity_prompt.append(entity_type)

entity_prompt.append(model.sep_token)

prompt_entity_length = len(entity_prompt)

# add prompt
tokens_p = [entity_prompt + tokens for tokens in x["tokens"]]
seq_length_p = x["seq_length"] + prompt_entity_length

out = model.token_rep_layer(tokens_p, seq_length_p)

word_rep_w_prompt = out["embeddings"]
mask_w_prompt = out["mask"]

word_rep = word_rep_w_prompt[:, prompt_entity_length:, :]
mask = mask_w_prompt[:, prompt_entity_length:]

# get_entity_type_rep
entity_type_rep = word_rep_w_prompt[:, :prompt_entity_length - 1, :]
# extract [ENT] tokens (which are at even positions in entity_type_rep)
entity_type_rep = entity_type_rep[:, 0::2, :]



In [119]:
tokens_p, seq_length_p, word_rep_w_prompt.shape

([['<<ENT>>',
   'name',
   '<<ENT>>',
   'city location far away from home',
   '<<SEP>>',
   'Hellow',
   'fedorov',
   'artem',
   ',',
   'how',
   'are',
   'you',
   '?']],
 tensor([13]),
 torch.Size([1, 13, 768]))

In [115]:
entity_type_rep

tensor([[[-0.0586, -0.3570, -0.1945,  ..., -0.6190,  0.1706, -0.5615],
         [ 0.2562, -0.4454,  0.4929,  ..., -0.0466,  0.5832, -0.2870]]],
       grad_fn=<SliceBackward0>)

In [90]:
word_rep_w_prompt.shape

torch.Size([1, 13, 768])

In [102]:
word_rep.shape

torch.Size([1, 8, 768])

In [122]:
print(inspect.getsource(model.forward))

    def forward(self, x):
        # compute span representation
        scores, num_classes, entity_type_mask = self.compute_score_train(x)
        batch_size = scores.shape[0]

        # loss for filtering classifier
        logits_label = scores.view(-1, num_classes)
        labels = x["span_label"].view(-1)  # (batch_size * num_spans)
        mask_label = labels != -1  # (batch_size * num_spans)
        labels.masked_fill_(~mask_label, 0)  # Set the labels of padding tokens to 0

        # one-hot encoding
        labels_one_hot = torch.zeros(labels.size(0), num_classes + 1, dtype=torch.float32).to(scores.device)
        labels_one_hot.scatter_(1, labels.unsqueeze(1), 1)  # Set the corresponding index to 1
        labels_one_hot = labels_one_hot[:, 1:]  # Remove the first column
        # Shape of labels_one_hot: (batch_size * num_spans, num_classes)

        # compute loss (without reduction)
        all_losses = F.binary_cross_entropy_with_logits(logits_label, labels_one_hot,
    

In [123]:
print(inspect.getsource(model.compute_score_train))

    def compute_score_train(self, x):
        span_idx = x["span_idx"] * x["span_mask"].unsqueeze(-1)

        new_length = x["seq_length"].clone()
        new_tokens = []
        all_len_prompt = []
        num_classes_all = []

        # add prompt to the tokens
        for i in range(len(x["tokens"])):
            all_types_i = list(x["classes_to_id"][i].keys())
            # multiple entity types in all_types. Prompt is appended at the start of tokens
            entity_prompt = []
            num_classes_all.append(len(all_types_i))
            # add enity types to prompt
            for entity_type in all_types_i:
                entity_prompt.append(self.entity_token)  # [ENT] token
                entity_prompt.append(entity_type)  # entity type
            entity_prompt.append(self.sep_token)  # [SEP] token

            # prompt format:
            # [ENT] entity_type [ENT] entity_type ... [ENT] entity_type [SEP]

            # add prompt to the tokens
            tokens_p = e

In [3]:
from matplotlib import pyplot as plt
import json
import io

from torch.utils.data.dataset import Dataset

In [4]:
class CultCode(Dataset):
    def __init__(self, train_on_single_tags=False, ):
        cult_code = json.load(io.open('./datasets/related/cult_code.json'))
        markups = cult_code['dataset']['markups']

        self.texts = []
        self.uno_span = []
        self.multi_span = []
        for i, mrkp in enumerate(markups):
            self.texts.append(mrkp['text'])
            
            for rel in mrkp['relations']:
                if train_on_single_tags and len(rel['tags']) > 1:
                    continue
                
                if len(rel['spans']) == 1:
                    begin = mrkp['spans'][rel['spans'][0]]['begin']
                    end   = mrkp['spans'][rel['spans'][0]]['end']
                    self.uno_span.append([i, [begin, end]])
                
                elif len(rel['spans']) > 1:
                    mult = []
                    for span in rel['spans']:
                        begin = mrkp['spans'][span]['begin']
                        end   = mrkp['spans'][span]['end']
                        mult.append([begin, end])
                    self.multi_span.append((i, mult))
    
    def get_len_eval(self):
        return len(self.multi_span)
    
    def get_item_eval(self, index):
        i, span = self.multi_span[index]
        return {
            'text': self.texts[i],
            'span': span
        }
    
    def __len__(self):
        return len(self.uno_span)
    
    def __getitem__(self, index):
        i, span = self.uno_span[index]
        return {
            'text': self.texts[i],
            'span': span
        }

In [7]:
len(CultCode(train_on_single_tags=True))

110

In [9]:
len(CultCode(train_on_single_tags=False))

2094